## Entorno de recomendación: diseño

In [1]:
# HIDDEN
import gym

In [2]:
# TODO: adapt images from Sven for this

#### Simular el comportamiento del usuario

- Simular el comportamiento del usuario al responder **repetidamente** a las recomendaciones de artículos.
- El comportamiento clave a simular:

&gt; Recomendar artículos de "comida basura" es bueno a corto plazo, pero malo a largo plazo.

- Esto es completamente nuestra elección como diseñador del entorno 
- Puede que quieras simular/captar un tipo diferente de comportamiento del usuario.
  - O bien, aprender de los datos del comportamiento del usuario -¡más adelante se hablará de esto!
- Pero éste será nuestro ejemplo en marcha por ahora.

#### Chocolate

- Modelaremos cada artículo como si tuviera un nivel de "dulzura" 
- Nos referiremos a los artículos de alto nivel de dulzura como "dulces".

![](img/candy.jpg)

- Pueden ser vídeos cortos y tontos, o baratijas baratas en oferta, etc.
- A los usuarios les encantan los caramelos a corto plazo, pero demasiado chocolate provoca insatisfacción a largo plazo.

#### Verduras

- Por otro lado, nos referiremos a los artículos poco dulces como "vegetales".

![](img/veggies.jpg)

- Podrían ser documentales educativos, o artículos aburridos pero útiles, etc.
- Los usuarios no disfrutan mucho de las verduras a corto plazo, pero aumentan la satisfacción a largo plazo.

#### Nivel de azúcar

- Modelaremos a nuestros usuarios como si tuvieran un **nivel de azúcar** que mide la cantidad de dulces que han comido recientemente.
- El nivel de azúcar del usuario (o la noción de nivel de azúcar) _no será conocido por el agente_
- Pero por ahora, estamos diseñando el simulador, así que lo conocemos todo.

#### Dinámica del nivel de azúcar

- Tenemos que decidir cómo cambia el nivel de azúcar con el consumo de artículos.
- Un enfoque sencillo es:

&gt; Cuando se consume un artículo, el nivel de azúcar se desplaza hacia la dulzura de ese artículo.

Ejemplos:

- Si tu nivel de azúcar es 0,2 y consumes un elemento con dulzor 0,5, tu nivel de azúcar sube ⬆️
- Si tu nivel de azúcar es 0,2 y consumes un artículo con dulzor 0,1, tu nivel de azúcar baja ⬇️

#### Dinámica del nivel de azúcar

- ¿Cómo representamos esto matemáticamente?
- Podemos intentar esto:

&gt; nuevo nivel de azúcar = ⍺ (antiguo nivel de azúcar) + (1 - ⍺) (dulzor del artículo)

- Aquí, ⍺ es un número entre 0 y 1 que controla lo "obstinado" que es el nivel de azúcar.

In [3]:
# HIDDEN 
# note the slide above and below are partially the same - just want to hide the bottom half at first

#### Dinámica del nivel de azúcar

- ¿Cómo representamos esto matemáticamente?
- Podemos intentar esto:

&gt; nuevo nivel de azúcar = ⍺ (antiguo nivel de azúcar) + (1 - ⍺) (dulzor del artículo)

- Aquí, ⍺ es un número entre 0 y 1 que controla lo "obstinado" que es el nivel de azúcar.
- Por ejemplo, si ⍺=1, la ecuación anterior se convierte en

&gt; nuevo nivel de azúcar = antiguo nivel de azúcar

y el nivel de azúcar nunca cambia. Si ⍺=0, tenemos

&gt; nuevo nivel de azúcar = dulzor del artículo

lo que significa que un solo artículo puede cambiar completamente el nivel de azúcar del usuario.

- Para ⍺ entre 0 y 1, tenemos una combinación del antiguo nivel de azúcar y la dulzura del artículo.

#### Dinámica del nivel de azúcar

Podemos poner en práctica lo anterior utilizando esta función:

In [4]:
def update_sugar_level(sugar_level, item_sweetness, alpha=0.9):
    return alpha * sugar_level + (1 - alpha) * item_sweetness

Vamos a probarlo para asegurarnos de que el comportamiento tiene sentido (utilizando el valor por defecto de ⍺=0,9):

In [5]:
sugar_level = 0.2
sugar_level = update_sugar_level(sugar_level, 0.8)
sugar_level

0.26

El artículo era dulce (0,8), así que el nivel de azúcar subió bastante.

In [6]:
sugar_level = update_sugar_level(sugar_level, 0.3)
sugar_level

0.264

La dulzura del artículo estaba ligeramente por encima del nivel de azúcar, por lo que el nivel de azúcar subió ligeramente.

#### Dinámica del nivel de azúcar

In [7]:
sugar_level = update_sugar_level(sugar_level, 0.01)
sugar_level

0.2386

El artículo no era dulce, por lo que el nivel de azúcar bajó.

#### Efecto de alfa

Podemos ver que, con un alfa más pequeño, el nivel de azúcar cambia mucho más rápido:

In [8]:
sugar_level = update_sugar_level(sugar_level, 0.0, alpha=0.5)
sugar_level

0.1193

#### Recompensa

- Muy bien, ¡ya tenemos resuelta la dinámica del nivel de azúcar!
- La segunda pieza importante del puzzle es la recompensa.
- Lo que queremos:

1. Un mayor nivel de dulzor en el artículo conlleva una mayor recompensa (¡dulces!)
2. Un mayor nivel de azúcar lleva a una menor recompensa (¡ah, demasiado caramelo!)

Una forma sencilla de combinar estos efectos es multiplicarlos:

&gt; recompensa = dulzura del objeto * (1 - nivel de azúcar)

#### Aplicación de la recompensa

&gt; recompensa = dulzura del objeto * (1 - nivel de azúcar)

Podemos codificar esto como

In [9]:
def reward(sugar_level, item_sweetness):
    return item_sweetness * (1 - sugar_level)

¡Utilizaremos estas piezas en la próxima sección cuando implementemos nuestro entorno!

#### Espacio de observación

- A continuación, tendremos que establecer las observaciones 
- Nuestras observaciones serán las _características de los artículos candidatos_.
- Para simplificar, supondremos sólo una característica, la dulzura del artículo.
- Así, el agente verá un montón de niveles de dulzura, y elegirá uno de ellos.

#### Espacio de acción

- En este entorno, la acción es el elemento elegido para recomendar, dados los canidatos.

#### ¡Apliquemos lo que hemos aprendido!

## Panorama general
<!-- multiple choice -->

¿Cuál de las siguientes opciones es **NO** cierta sobre el entorno de recomendación RL simulado que estamos creando?

- [ ] El entorno contiene un modelo muy simplificado del comportamiento de los usuarios, pero un agente entrenado aún puede ser útil para hacer recomendaciones.
- [ ] El entorno representa con exactitud cómo se comportan los usuarios reales.
- [ ] El entorno es un buen punto de partida, y es posible que queramos añadir complejidad a medida que avanza nuestro trabajo.
- [ ] El entorno capta la noción de que los usuarios responderán de forma diferente a los distintos elementos, y esta respuesta puede depender de su historial.

## Recompensas del recomendador
<!-- multiple choice -->

Recordemos que nuestra función de recompensa es

&gt; recompensa = dulzura del artículo * (1 - nivel de azúcar)

#### Satisfacción a corto plazo

Verdadero o falso: en un momento dado, la recompensa _inmediata_ es _siempre_ mayor para los dulces que para las verduras.

- [x] Verdadero | La recompensa inmediata es directamente proporcional a la dulzura del artículo.
- [ ] Falso | ¡Mira bien la fórmula de arriba!

#### Satisfacción a largo plazo

Verdadero o falso: en un momento dado, la recompensa total _a largo plazo_ es _siempre_ mayor por recomendar verduras que por dulces.

- [] Verdadero: es complicado determinar qué será lo mejor a largo plazo, ¡esto es lo que tiene que aprender nuestro agente!
- [x] Falso

## Choque de azúcar
<!-- coding exercise -->

Supongamos que tu nivel de azúcar comienza en 0,5, y que en cada paso sólo tienes dos elementos para elegir, megaverduras (dulzura = 0) y megacaramelos (dulzura = 1). Harás 3 recomendaciones seguidas, utilizando alfa = 0,7. Utiliza la ventana de codificación de abajo para jugar con diferentes opciones y encontrar la mejor secuencia de recomendaciones en términos de recompensa _total_ 

In [10]:
# EXERCISE

def update_sugar_level(sugar_level, item_sweetness, alpha=0.9):
    return alpha * sugar_level + (1 - alpha) * item_sweetness

def reward(sugar_level, item_sweetness):
    return item_sweetness * (1 - sugar_level)

# MODIFY THIS LIST
# But make sure it always contains 3 items, each 0 or 1
recommendations = [0, 0, 0]

# starting sugar level
sugar_level = 0.5

total_reward = 0

for item_sweetness in recommendations:
    
    # add reward
    immediate_reward = reward(sugar_level, item_sweetness)
    total_reward += immediate_reward
    
    # update sugar level
    sugar_level = update_sugar_level(sugar_level, item_sweetness, alpha=0.7)
    
    print(f"  Received reward {immediate_reward:.5f}, new sugar level {sugar_level:.5f}")
    
print("Total reward after 5 recommendations:", total_reward)

  Received reward 0.00000, new sugar level 0.35000
  Received reward 0.00000, new sugar level 0.24500
  Received reward 0.00000, new sugar level 0.17150
Total reward after 5 recommendations: 0.0


In [11]:
# SOLUTION

def update_sugar_level(sugar_level, item_sweetness, alpha=0.9):
    return alpha * sugar_level + (1 - alpha) * item_sweetness

def reward(sugar_level, item_sweetness):
    return item_sweetness * (1 - sugar_level)

# MODIFY THIS LIST
# But make sure it always contains 3 items, each 0 or 1
recommendations = [0,1,1]

# starting sugar level
sugar_level = 0.5

total_reward = 0

for item_sweetness in recommendations:
    
    # add reward
    immediate_reward = reward(sugar_level, item_sweetness)
    total_reward += immediate_reward
    
    # update sugar level
    sugar_level = update_sugar_level(sugar_level, item_sweetness, alpha=0.7)
    
    print(f"  Received reward {immediate_reward:.5f}, new sugar level {sugar_level:.5f}")
    
print("Total reward after 5 recommendations", total_reward)

  Received reward 0.00000, new sugar level 0.35000
  Received reward 0.65000, new sugar level 0.54500
  Received reward 0.45500, new sugar level 0.68150
Total reward after 5 recommendations 1.105


#### ¿Cuál fue la mejor estrategia en este ejemplo?

- [x] 1 verdura para bajar los niveles de azúcar, luego 2 caramelos para esa dulce recompensa.
- [ ] Caramelos, luego verduras para tener buena salud, luego más caramelos.
- [ ] ¡Vegetales hasta el final!
- [ ] ¡Caramelos hasta el final!